# CHAPTER 10 Data Aggregation and Group Operations

__What is goup operations? _split-apply-combine___

`df['data'].groupby(['col_name_1','col_name_2'])`

- `groupby()` returns a Groupby object, which contains all the information needed to then apply some operations to each of the groups
- Notice when apply some functions (such as `.mean()`), the index of the result are exactly the things you passed to the Groupby object. If you pass a list of columns to `groupby()` then the final output's index will be hierachical.
- Iterating: Groupby object support iterating. `for name,group in df.groupby('key1')`
- Use functions to group: `.groupby(func, as_index=False)` will invoke function on __each index value__ and use the return value as the new index.
- Invoke functions into Groupby object, such as `.mean()`. 
    - `agg`. It's also available for own functions: **notice the parameter should be an `array`.**
        ```python
        def peak_to_peak(arr):
            return arr.man()-arr.min()

        object.agg(peak_to_peak)
        ```
    - `agg()` is able to receive dictionary, list, etc.
- `apply()`
    - When used upon Groupby object, it invokes on each "row group" from the DataFrame and then the results are glued together using `pandas.concat`, labeling the pieces with the group names.
    ```python
        def top(df,n=5,column='tip_pct'):
            return df.sort_values(by=column)[-n:]

        tips.groupby(['smoker','day']).apply(top,n=2,column='total_bill')
    ```
   
    - return of `pd.cut()` and `pd.qcut()` can be invoked into Groupby object `apply()`
    - With `.groupby().apply(lambda g: g.fillna(g.mean())` you can fill missing data intrinsiclly in groups
    
- Pivot table
    - `tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker',margins=False,aggfunc=,fill_value=)`
- Cross-Tabulations
    `pd.crosstab([tips['time'],tips['day']],tips['smoker'],margins=True)`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
df = pd.DataFrame({
    'key1':list('aabba'),
    'key2':['one','two','one','two','one'],
    'data1':np.random.randn(5),
    'data2':np.random.randn(5)
})
df

,key1,key2,data1,data2
0,a,one,-1.322504,-1.438196
1,a,two,-1.222986,1.389363
2,b,one,-0.868646,-0.913017
3,b,two,-0.436393,-0.253564
4,a,one,1.856276,0.031706


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()
# notice that the name of index is 'key1'

key1
a   -0.229738
b   -0.652520
Name: data1, dtype: float64

In [5]:
grouped = df['data1'].groupby([df['key1'],df['key2']])
grouped

In [6]:
grouped.mean()

key1  key2
a     one     0.266886
      two    -1.222986
b     one    -0.868646
      two    -0.436393
Name: data1, dtype: float64

In [7]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

California  2005   -1.222986
            2006   -0.868646
Ohio        2005   -0.879449
            2006    1.856276
Name: data1, dtype: float64

In [8]:
df

,key1,key2,data1,data2
0,a,one,-1.322504,-1.438196
1,a,two,-1.222986,1.389363
2,b,one,-0.868646,-0.913017
3,b,two,-0.436393,-0.253564
4,a,one,1.856276,0.031706


In [9]:
means = df.groupby(['key1','key2']).mean()
means

data1     data2
key1 key2                    
a    one   0.266886 -0.703245
     two  -1.222986  1.389363
b    one  -0.868646 -0.913017
     two  -0.436393 -0.253564

In [10]:
means.unstack()

data1               data2          
key2       one       two       one       two
key1                                        
a     0.266886 -1.222986 -0.703245  1.389363
b    -0.868646 -0.436393 -0.913017 -0.253564

In [11]:
df

,key1,key2,data1,data2
0,a,one,-1.322504,-1.438196
1,a,two,-1.222986,1.389363
2,b,one,-0.868646,-0.913017
3,b,two,-0.436393,-0.253564
4,a,one,1.856276,0.031706


In [12]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -1.322504 -1.438196
1    a  two -1.222986  1.389363
4    a  one  1.856276  0.031706
b
  key1 key2     data1     data2
2    b  one -0.868646 -0.913017
3    b  two -0.436393 -0.253564


In [13]:
for (k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -1.322504 -1.438196
4    a  one  1.856276  0.031706
('a', 'two')
  key1 key2     data1     data2
1    a  two -1.222986  1.389363
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.868646 -0.913017
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.436393 -0.253564


In [14]:
df

,key1,key2,data1,data2
0,a,one,-1.322504,-1.438196
1,a,two,-1.222986,1.389363
2,b,one,-0.868646,-0.913017
3,b,two,-0.436393,-0.253564
4,a,one,1.856276,0.031706


In [15]:
df['data1']

0   -1.322504
1   -1.222986
2   -0.868646
3   -0.436393
4    1.856276
Name: data1, dtype: float64

In [16]:
df[['data1']]

,data1
0,-1.322504
1,-1.222986
2,-0.868646
3,-0.436393
4,1.856276


In [17]:
for name,group in df.groupby('key1')['data1']:
    print(name)
    print(group)

a
0   -1.322504
1   -1.222986
4    1.856276
Name: data1, dtype: float64
b
2   -0.868646
3   -0.436393
Name: data1, dtype: float64


In [18]:
df

,key1,key2,data1,data2
0,a,one,-1.322504,-1.438196
1,a,two,-1.222986,1.389363
2,b,one,-0.868646,-0.913017
3,b,two,-0.436393,-0.253564
4,a,one,1.856276,0.031706


In [19]:
df.groupby(['key1','key2'])[['data1']].mean()

data1
key1 key2          
a    one   0.266886
     two  -1.222986
b    one  -0.868646
     two  -0.436393

In [20]:
people = pd.DataFrame(np.random.randn(5, 5),columns=['a', 'b', 'c', 'd', 'e'],index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [21]:
people

,a,b,c,d,e
Joe,0.772381,-1.448307,-0.731343,0.165859,-0.000316
Steve,-0.178851,-0.309901,0.187103,-0.463590,0.394812
Wes,-0.556368,1.497646,-1.194583,-0.229090,0.359712
Jim,-0.729397,0.556021,0.514672,-0.443149,0.918638
Travis,2.592350,-0.086965,0.488818,-0.340815,-2.274177


In [22]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.513384,0.605360,-1.411254,-0.506379,1.278034
5,-0.178851,-0.309901,0.187103,-0.463590,0.394812
6,2.592350,-0.086965,0.488818,-0.340815,-2.274177


In [23]:
df

,key1,key2,data1,data2
0,a,one,-1.322504,-1.438196
1,a,two,-1.222986,1.389363
2,b,one,-0.868646,-0.913017
3,b,two,-0.436393,-0.253564
4,a,one,1.856276,0.031706


In [25]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.240424
b   -0.479618
Name: data1, dtype: float64

In [44]:
def peak_to_peak(arr):
    return arr.max()-arr.min()

grouped.agg(peak_to_peak)

/var/folders/7l/j9np0sbx1b33249mt44v25980000gn/T/ipykernel_1794/3391053486.py:4: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  grouped.agg(peak_to_peak)


total_bill   tip  size   tip_pct
day  smoker                                  
Fri  No           10.29  2.00     1  0.067349
     Yes          34.42  3.73     3  0.159925
Sat  No           41.08  8.00     3  0.235193
     Yes          47.74  9.00     4  0.290095
Sun  No           39.40  4.99     4  0.193226
     Yes          38.10  5.00     3  0.644685
Thur No           33.68  5.45     5  0.193350
     Yes          32.77  3.00     2  0.151240

In [27]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.229738  1.807227 -1.322504 -1.272745 -1.222986  0.316645   
b      2.0 -0.652520  0.305649 -0.868646 -0.760583 -0.652520 -0.544456   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.856276   3.0 -0.005709  1.414151 -1.438196 -0.703245  0.031706   
b    -0.436393   2.0 -0.583291  0.466304 -0.913017 -0.748154 -0.583291   

                          
           75%       max  
key1                      
a     0.710534  1.389363  
b    -0.418427 -0.253564

In [38]:
tips = pd.read_csv('tips.csv')
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [39]:
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [41]:
grouped = tips.groupby(['day','smoker'])

In [42]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [46]:
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [49]:
results = grouped['tip_pct','total_bill'].agg(['count','mean','max'])
results

/var/folders/7l/j9np0sbx1b33249mt44v25980000gn/T/ipykernel_1794/1349474437.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  results = grouped['tip_pct','total_bill'].agg(['count','mean','max'])


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [50]:
def top(df,n=5,column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips,n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [51]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [53]:
tips.groupby(['smoker','day']).apply(top,n=2,column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  91        22.49   3.50     No   Fri  Dinner     2  0.155625
            94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  59        48.27   6.73     No   Sat  Dinner     4  0.139424
            212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  112       38.07   4.00     No   Sun  Dinner     3  0.105070
            156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 85        34.83   5.17     No  Thur   Lunch     4  0.148435
            142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
            95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
            170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
            182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 83        32.68   5.00    Yes  Thur   Lunch     2  0.152999
            197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [54]:
frame = pd.DataFrame({
    'data1':np.random.randn(1000),
    'data2':np.random.randn(1000)
})
frame

,data1,data2
0,1.200864,-0.168146
1,-0.469546,-1.752359
2,0.580097,1.145352
3,-0.779408,0.754534
4,-0.963783,0.011992
...,...,...
995,-2.037915,1.408718
996,0.593526,0.251092
997,-0.724831,-2.144624
998,-0.116128,-1.363521


In [55]:
quartiles = pd.cut(frame['data1'],4)
quartiles

0         (0.28, 1.977]
1        (-1.417, 0.28]
2         (0.28, 1.977]
3        (-1.417, 0.28]
4        (-1.417, 0.28]
             ...       
995    (-3.121, -1.417]
996       (0.28, 1.977]
997      (-1.417, 0.28]
998      (-1.417, 0.28]
999       (0.28, 1.977]
Name: data1, Length: 1000, dtype: category
Categories (4, interval[float64, right]): [(-3.121, -1.417] < (-1.417, 0.28] < (0.28, 1.977] < (1.977, 3.673]]

In [61]:
def get_stats(group):
    return {
        'min':group.min(),
        'max':group.max(),
        'count':group.count(),
        'mean':group.mean()
    }

grouped = frame['data2'].groupby(quartiles)
grouped.apply(get_stats)

data1                  
(-3.121, -1.417]  min       -2.456689
                  max        2.462809
                  count     79.000000
                  mean       0.144009
(-1.417, 0.28]    min       -2.376552
                  max        2.502012
                  count    497.000000
                  mean      -0.039433
(0.28, 1.977]     min       -2.862433
                  max        3.344276
                  count    399.000000
                  mean       0.103552
(1.977, 3.673]    min       -2.902669
                  max        1.848781
                  count     25.000000
                  mean      -0.276821
Name: data2, dtype: float64

In [67]:
grouping = pd.qcut(frame['data1'],10)
grouped = frame['data2'].groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.1149999999999998, -1.287]",-2.456689,2.462809,100.0,0.116419
"(-1.287, -0.798]",-2.343581,2.460245,100.0,-0.170963
"(-0.798, -0.433]",-2.144624,1.815485,100.0,-0.049401
"(-0.433, -0.167]",-2.077813,2.502012,100.0,0.018308
"(-0.167, 0.0841]",-2.241673,2.050100,100.0,-0.097655
"(0.0841, 0.339]",-2.376552,2.604594,100.0,0.140737
"(0.339, 0.594]",-2.862433,2.474383,100.0,0.018588
"(0.594, 0.886]",-2.412883,2.284355,100.0,0.159583
"(0.886, 1.23]",-2.516291,3.278911,100.0,0.176520


In [70]:
s = pd.Series(np.random.randn(6))
s[::2]=np.nan
s

0         NaN
1    0.558301
2         NaN
3    1.942556
4         NaN
5   -0.231390
dtype: float64

In [71]:
s.fillna(s.mean())

0    0.756489
1    0.558301
2    0.756489
3    1.942556
4    0.756489
5   -0.231390
dtype: float64

In [72]:
states = ['Ohio','New York','Vermont','Florida','Oregon','Nevada','California','Idaho']
group_key = ['East']*4+['West']*4
data = pd.Series(np.random.randn(8),index=states)
data

Ohio         -0.059941
New York      2.373834
Vermont      -0.003563
Florida      -1.257289
Oregon       -0.581785
Nevada        0.313082
California    0.446267
Idaho         1.266113
dtype: float64

In [73]:
data[['Vermont','Nevada','Idaho']]=np.nan
data

Ohio         -0.059941
New York      2.373834
Vermont            NaN
Florida      -1.257289
Oregon       -0.581785
Nevada             NaN
California    0.446267
Idaho              NaN
dtype: float64

In [75]:
data.groupby(group_key).mean()

East    0.352201
West   -0.067759
dtype: float64

In [77]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -0.059941
New York      2.373834
Vermont       0.352201
Florida      -1.257289
Oregon       -0.581785
Nevada       -0.067759
California    0.446267
Idaho        -0.067759
dtype: float64

In [81]:
suits = list('HSCD')
card_val = (list(range(1,11))+[10]*3)*4
base_names = ['A']+list(range(2,11))+['J','K','Q']
cards=[]
for suit in suits:
    cards.extend(str(num)+suit for num in base_names)
deck = pd.Series(card_val, index=cards)

In [84]:
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [85]:
def draw(deck,n=5):
    return deck.sample(n)
draw(deck)

4C    4
8C    8
8S    8
6C    6
2D    2
dtype: int64

In [86]:
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw,n=2)

C  10C    10
   4C      4
D  JD     10
   3D      3
H  2H      2
   4H      4
S  9S      9
   8S      8
dtype: int64

In [87]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
             'b', 'b', 'b', 'b'],
'data': np.random.randn(8),
'weights': np.random.rand(8)})

In [88]:
df

,category,data,weights
0,a,0.765600,0.510635
1,a,1.799632,0.791987
2,a,0.548114,0.865542
3,a,0.361839,0.433104
4,b,-1.077980,0.367270
5,b,0.561070,0.691268
6,b,1.009312,0.556837
7,b,0.546225,0.788981


In [89]:
grouped = df.groupby('category')

In [90]:
get_wavg = lambda g:np.average(g['data'],weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.940832
b    0.409641
dtype: float64

In [92]:
close_px = pd.read_csv('/Users/yanyan/Documents/MyCode/《Python for Data Analysis》学习与练习/pydata-book-3rd-edition/examples/stock_px.csv', parse_dates=True,index_col=0)

In [93]:
close_px

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93
...,...,...,...,...
2011-10-10,388.81,26.94,76.28,1194.89
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66


In [94]:
spx_corr = lambda x: x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()
rets

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
...,...,...,...,...
2011-10-10,0.051406,0.026286,0.036977,0.034125
2011-10-11,0.029526,0.002227,-0.000131,0.000544
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974


In [95]:
get_year = lambda x:x.year
by_year = rets.groupby(get_year)

In [96]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [98]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [100]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [101]:
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [114]:
tips.pivot_table(['tip_pct','size'],index=['time','day'],columns='smoker',margins=True,aggfunc=len)

size            tip_pct           
smoker          No   Yes  All      No   Yes  All
time   day                                      
Dinner Fri     3.0   9.0   12     3.0   9.0   12
       Sat    45.0  42.0   87    45.0  42.0   87
       Sun    57.0  19.0   76    57.0  19.0   76
       Thur    1.0   NaN    1     1.0   NaN    1
Lunch  Fri     1.0   6.0    7     1.0   6.0    7
       Thur   44.0  17.0   61    44.0  17.0   61
All          151.0  93.0  244   151.0  93.0  244

In [115]:
pd.crosstab([tips['time'],tips['day']],tips['smoker'],margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244